In [ ]:
# importing necessary libraries

import nltk, re, string 
# https://www.nltk.org (natural language toolkit)
# https://docs.python.org/3/library/re.html (regular expression toolkit)
# https://docs.python.org/3/library/string.html

from nltk.corpus import stopwords, twitter_samples
import numpy as np
import pickle
# https://docs.python.org/3/library/pickle.html
# For better understanding of Logistic Regression: https://towardsdatascience.com/introduction-to-logistic-regression-66248243c148

In [ ]:
# Preprocessing our data (tweets, reviews, messages, etc.):

#When preprocessing, you have to perform the following:
#Eliminate handles and URLs
#Tokenize the string into words.
#Remove stop words like “and, is, a, on, etc.”
#Stemming- or convert every word to its stem. Like a dancer, dancing, danced, becomes ‘danc’. You can use porter stemmer to take care of this.
#Convert all your words to lower case.


def process_text(tweet):   # specifically for twitter (see below)
    stemmer = nltk.PorterStemmer()       # define the stemming command
    stopwords_english = stopwords.words('english')
    tweet = re.sub(r'\$\w*', '', tweet)
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub(r'#', '', tweet)

    tokenizer = nltk.TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)

    # https://www.nltk.org/api/nltk.tokenize.casual.html
    # Remove handles (twitter usernames: https://sproutsocial.com/insights/twitter-handle/)
    # Reduce len: . By default, it is set to False. It specifies whether to replace repeated character sequences of length 3 or greater with sequences of length 3
    # Example: "!!!!" -> "!", "!", "!", see: https://www.nltk.org/api/nltk.tokenize.casual.html

    tweet_tokens = tokenizer.tokenize(tweet)
    # use the function "tozenize" of the library "tokenizer".
    # The library "tokenizer" is exactly "nltk.TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)", defined above
    # from the nltk package

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and
                word not in string.punctuation):
            stem_word = stemmer.stem(word)  # use the stemming command (defined above)
            tweets_clean.append(stem_word)

    return tweets_clean

In [ ]:
# this function will be needed later

def build_freqs(tweets, ys):
    """Build frequencies.
    Input:
        tweets: a list of tweets
        ys: an m x 1 array with the sentiment label (positive or negative) of each tweet
            (either 0 or 1)
    Output:
        freqs: a dictionary mapping each (word, sentiment) pair to its
        frequency
    """
    # Convert np array to list since zip (see below) needs an iterable.
    # The squeeze is necessary or the list ends up with one element.
    # Also note that this is non necessary if ys is already a list.
    # Check the usage of "sqeeze" with the following command:
    #yslist.shape
    yslist = np.squeeze(ys).tolist()

    # Start with an empty dictionary and populate it by looping over all tweets
    # and over all processed words in each tweet.
    # Check the usage of zip: https://www.programiz.com/python-programming/methods/built-in/zip

    nltk.download('stopwords')
    freqs = {}
    for y, tweet in zip(yslist, tweets):
        for word in process_text(tweet): # this will need a dictionary of stop words (used into the "process_text" function)
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1 # if already in "freqs" then increase by 1 its frequency
            else:
                freqs[pair] = 1 # otherwise just put frequency = 1 (first time this pair appears)

    return freqs

In [ ]:



# Checking how the above code works with an example.

tweets = ['i am happy', 'i am tricked', 'i am sad', 'i am tired', 'i am tired']
ys = [1, 0, 0, 0, 0]
res = build_freqs(tweets, ys)
print(res)

{('happi', 1): 1, ('trick', 0): 1, ('sad', 0): 1, ('tire', 0): 2}


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
nltk.download('twitter_samples')

# select the set of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')



# split the data into two pieces, one for training and one for testing.
# look at this cool way to split the data

test_pos = all_positive_tweets[4000:] 
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]


train_x = train_pos + train_neg
test_x = test_pos + test_neg

# combine positive and negative labels
# We are building our y - target variable here
# since it's a binary problem (0, 1), building the labels is very easy:
# 1 and 0 according to the fact the sentiment is positive or negative
# len(train-pos) tells you how many 1 you have to put

train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

# np.ones((len(train_pos), 1) : shape of the array: Nx1

# create frequency dictionary
#remember the syntax (see above): tweets, ylabels
freqs = build_freqs(train_x, train_y)



[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# check out the output

print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 11338


In [ ]:
# test the function below

print('This is an example of a positive tweet: \n', train_x[22])
print('\nThis is an example of the processed version of the tweet: \n', process_text(train_x[22]))

This is an example of a positive tweet: 
 @gculloty87 Yeah I suppose she was lol! Chat in a bit just off out x :))

This is an example of the processed version of the tweet: 
 ['yeah', 'suppos', 'lol', 'chat', 'bit', 'x', ':)']


In [ ]:
# In this case, we will actully build a Logistic Regression Model from scratch
#  Logistic regression

# Sigmoid Function
#The sigmoid function gives values between -1 and 1 
#hence we can classify the predictions depending on a particular cutoff. (say : 0.5)

def sigmoid(z):
    """
    Input:
        z: is the input (can be a scalar or an array)
    Output:
        h: the sigmoid of z
    """
    zz = np.negative(z)
    h = 1 / (1 + np.exp(zz))
    return h

In [ ]:
# Cost function and Gradient

#You initialize your parameter θ, that you can use in your sigmoid, 
#you then compute the gradient that you will use to update θ, and then calculate the cost. 
#You keep doing so until good enough.

def gradientDescent(x, y, theta, alpha, num_iters):
    """
    Input:
        x: matrix of features which is (m,n+1)
        y: corresponding labels of the input matrix x, dimensions (m,1)
        theta: weight vector of dimension (n+1,1)
        alpha: learning rate
        num_iters: number of iterations you want to train your model for
    Output:
        J: the final cost
        theta: your final weight vector

    Hint: you might want to print the cost to make sure that it is going down.
    """
    # get 'm', the number of rows in matrix x
    m = x.shape[0]
    for i in range(0, num_iters):

        z = np.dot(x, theta)
        h = sigmoid(z)

        # calculate the cost function for the logistic regression:
       # theory can be found here: https://towardsdatascience.com/introduction-to-logistic-regression-66248243c148

       #the cost function represents optimization objective 
      #i.e. we create a cost function and minimize it so that we can develop an accurate model with minimum error.
 

        cost = -1. / m * (np.dot(y.transpose(), np.log(h)) + np.dot((1 - y).transpose(), np.log(1 - h)))

        # update the weights theta (alpha is the learning rate)
        theta = theta - (alpha / m) * np.dot(x.transpose(), (h - y))

    cost = float(cost)
    return cost, theta


In [ ]:
#  Extracting the features

def extract_features(tweet, freqs):
    """
    Input:
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output:
        x: a feature vector of dimension (1,3)
    """

    word_l = process_text(tweet)
    x = np.zeros((1, 3))

    # bias term is set to 1
    x[0, 0] = 1

    for word in word_l:
        # increment the word count for the positive label 1
        x[0, 1] += freqs.get((word, 1.0), 0)
        # increment the word count for the negative label 0
        x[0, 2] += freqs.get((word, 0.0), 0)

    assert (x.shape == (1, 3))
    return x

In [ ]:
# test on training data

tmp1 = extract_features(train_x[22], freqs)
print(tmp1)

# Try to understand what all these three numbers mean. 
# Usually we get a dataset with a lot of features/columns, here we just have text data.
# Those three numbers are the feature set that we have build using build_freq() and extract_features() function.
# build_freq() builds a dictionary having words as keys and the number of times they have occurred in corpus as values.
# Extract feature takes in sum of these values for positive and negative words, i.e. tmp1[1] and tmp[2]


# How these features will be used to predict in Logistic Regression

# First a hypothesis is build which for our case will be h(x) = b1 + b2*x1 + b3*x2
# here b1 = 1, b2 and b3 are determined by cost and gradient function, x1 and x2 are the positive and negative words feature set.

[[1.000e+00 3.006e+03 1.240e+02]]


In [ ]:
# Training Your Model

# collect the features 'x' and stack them into a matrix 'X'
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :] = extract_features(train_x[i], freqs)

# training labels corresponding to X
Y = train_y

# Apply gradient descent
# these values are predefined (Andrew NG)
J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 1500)

In [ ]:
def predict_tweet(tweet, freqs, theta):
    """
    Input:
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
    Output:
        y_pred: the probability of a tweet being positive or negative
    """
    # extract the features of the tweet and store it into x
    x = extract_features(tweet, freqs)
    y_pred = sigmoid(np.dot(x, theta))

    return y_pred


In [ ]:
def test_logistic_regression(test_x, test_y, freqs, theta):
    """
    Input:
        test_x: a list of tweets
        test_y: (m, 1) vector with the corresponding labels for the list of tweets
        freqs: a dictionary with the frequency of each pair (or tuple)
        theta: weight vector of dimension (3, 1)
    Output:
        accuracy: (# of tweets classified correctly) / (total # of tweets)
    """
    # the list for storing predictions
    y_hat = []

    for tweet in test_x:
        # get the label prediction for the tweet
        y_pred = predict_tweet(tweet, freqs, theta)
        if y_pred > 0.5:
            y_hat.append(1)
        else:
            y_hat.append(0)

    accuracy = (y_hat == np.squeeze(test_y)).sum() / len(test_x)

    return accuracy

In [ ]:
tmp_accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print(f"Logistic regression model's accuracy = {tmp_accuracy:.4f}")

Logistic regression model's accuracy = 0.9950


In [ ]:
# Predict with your own tweet

def pre(sentence):
    yhat = predict_tweet(sentence, freqs, theta)
    if yhat > 0.5:
        return 'Positive sentiment'
    elif yhat == 0:
        return 'Neutral sentiment'
    else:
        return 'Negative sentiment'

In [ ]:
my_tweet = 'This algorithm is pretty bad, but it was implemented correctly'

res = pre(my_tweet)
print(res)

Negative sentiment
